## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

## Loading Products Dataset

In [2]:
products_df = pd.read_csv('PRODUCTS_TAKEHOME.csv')
products_df.shape
products_df

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11
...,...,...,...,...,...,...,...
845547,Health & Wellness,Topical Muscle & Joint Relief Treatments,Braces & Wraps,NaN,NaN,NaN,7.223016e+11
845548,Snacks,Cookies,NaN,NaN,"TREEHOUSE FOODS, INC.",LOFTHOUSE,4.182082e+10
845549,Snacks,Candy,Confection Candy,NaN,HARIBO GMBH & CO KG,HARIBO,1.001672e+11
845550,Snacks,Nuts & Seeds,Hazelnuts,NaN,DOUBLE-COLA CO,JUMBO,7.539076e+10


In [3]:
#Familiarizing with Dataset
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845552 entries, 0 to 845551
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    845441 non-null  object 
 1   CATEGORY_2    844128 non-null  object 
 2   CATEGORY_3    784986 non-null  object 
 3   CATEGORY_4    67459 non-null   object 
 4   MANUFACTURER  619078 non-null  object 
 5   BRAND         619080 non-null  object 
 6   BARCODE       841527 non-null  float64
dtypes: float64(1), object(6)
memory usage: 45.2+ MB


## Checking the Dataset for the Missing and Duplicated Values.

In [4]:
products_df.duplicated().sum()

215

We only have 215 duplicated rows in the whole dataset. We can just remove those rows from our dataset before further data analysis

In [5]:
#Checking for Null Values
products_df.isnull().sum()

CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64

In [6]:
#Calculating the percentage of missing values for each column
missing_percentage = (products_df.isnull().sum() / len(products_df)) * 100
print(missing_percentage.apply(lambda x: f'{x:.2f}%'))

CATEGORY_1       0.01%
CATEGORY_2       0.17%
CATEGORY_3       7.16%
CATEGORY_4      92.02%
MANUFACTURER    26.78%
BRAND           26.78%
BARCODE          0.48%
dtype: object


As we can see, very large percentage of Brand and Manufacturer is missing. It is a huge data quality issue.

We can also see that Category_4 has a extremely high percentage of missing values and Category_3 also has a significant amount of records that are missing. Based on the data exploration, it can be understood that there is a hierarchy of categories with Category_1 as the general domain of the product and Category_4 as the most relevant sub category of the product. Assuming that it is not possible to drill down every product to the last subcategory, the missing data will not be considered as a major data quality issue.

Identifying the user favorite Brands and Manufacturers

In [7]:
top_manufacturers = products_df.groupby('MANUFACTURER')['BARCODE'].count().nlargest(5)
for manufacturer in top_manufacturers.index:
    top_brands = products_df[products_df['MANUFACTURER'] == manufacturer].groupby('BRAND')['BARCODE'].count().nlargest(5)
    print(f"\nManufacturer: {manufacturer}")
    print(top_brands.to_string())


Manufacturer: PLACEHOLDER MANUFACTURER
BRAND
BRAND NOT KNOWN       17025
PRIVATE LABEL         13467
BRAND NEEDS REVIEW     2904
PAUL MITCHELL          2004
RUSSELL STOVER         1250

Manufacturer: REM MANUFACTURER
BRAND
REM BRAND    20813

Manufacturer: PROCTER & GAMBLE
BRAND
PANTENE      2124
OLAY         1922
OLD SPICE    1822
CREST        1796
ALWAYS       1582

Manufacturer: L'OREAL
BRAND
MATRIX                      2958
REDKEN                      2579
L'OREAL PARIS HAIR COLOR    1949
L'OREAL PARIS HAIR CARE     1519
GARNIER HAIR CARE           1480

Manufacturer: UNILEVER
BRAND
DOVE             3834
SHEA MOISTURE    2505
AXE              1889
TRESEMMÉ         1655
VASELINE         1280


Based on studying the results above, we can see that one of the top-selling manufacturers and brands has placeholder values, indicating incomplete or incorrect data. This is a major data quality issue, as it can skew business insights, lead to inaccurate reporting on top products, and hinder targeted marketing efforts and partnerships with legitimate brands.

# Final Thoughts:

The major data quality issues found in this dataset:

1) 26% of Values are missing for the Brand and Manufacturer columns. This is a major data quality issue as these are the important identifiers of any product. Without these it would be difficult for Fetch to personalise user experience by recommendations and it will also hinder in fetch's loyalty partnerships with brands (To decide what type of reward points to set in which product of which brands). Additionally, based on the above exploration, one of the top-selling manufacturers and brands has placeholder values, further highlighting the severity of this issue. This can lead to inaccurate business insights, misreporting of top products, and hindered partnerships with legitimate brands. Addressing this is crucial for accurate analysis and optimal customer engagement.

Minor Data Quality issues found in this dataset:

1) The product categories follow a hierarchical structure, with Category_1 representing the broad domain of the product and Category_4 being the most specific subcategory. A significant number of values are missing in Category_4, likely because it's difficult to classify certain products at such a detailed level. While missing values in Category_3 pose a minor data quality issue, the lower percentage of missing data suggests that most products are generally well-categorized, though a few may lack proper classification.

2) Small number of Redundant Data (Duplicate Rows), We can remove them to obtain a consistent dataset.

3) Small percentage of values are missing in Barcode Colums which will have to be dealt with.